<a href="https://colab.research.google.com/github/Manpreet102005/Diabetes-Prediction/blob/main/Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
 pip install opendatasets

In [3]:
import opendatasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import os

In [4]:
opendatasets.download("https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: manpreetsinghkashyap
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database


100%|██████████| 8.91k/8.91k [00:00<00:00, 29.0MB/s]

In [96]:
df=pd.read_csv("/content/pima-indians-diabetes-database/diabetes.csv")

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [98]:
from sklearn.model_selection import train_test_split
train_df_1,test_df=train_test_split(df,test_size=0.2,random_state=758)
train_df,val_df=train_test_split(train_df_1,test_size=0.25,random_state=758)

In [99]:
inputs=list(df.columns[:-1])
target="Outcome"

In [100]:
X_train=train_df[inputs].copy()
Y_train=train_df[target].copy()
X_val=val_df[inputs].copy()
Y_val=val_df[target].copy()
X_test=test_df[inputs].copy()
Y_test=test_df[target].copy()

In [101]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Since BP,Insulin,Glucose and SkinThickness can not be zero.
So replacing 0 entries with null and will impute them with suitable method

In [102]:
cols_0=['BloodPressure','Insulin','Glucose','SkinThickness']
df[cols_0]=df[cols_0].replace(0,np.nan)

In [103]:
df.isna().sum()

,0
Pregnancies,0
Glucose,5
BloodPressure,35
SkinThickness,227
Insulin,374
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [104]:
# ploting histogram for cols_0

In [105]:
glucose=px.histogram(df,'Glucose')
BP=px.histogram(df,'BloodPressure')
skin_thickness=px.histogram(df,'SkinThickness')
insulin=px.histogram(df,'Insulin')
glucose.show()
BP.show()
skin_thickness.show()
insulin.show()

In [106]:
# since all follows Normal Distribution, Values should be imputed with mean values
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='mean').fit(df[cols_0])
X_train[cols_0]=imputer.transform(X_train[cols_0])
X_val[cols_0]=imputer.transform(X_val[cols_0])
X_test[cols_0]=imputer.transform(X_test[cols_0])

In [109]:
X_train.isna().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0


In [110]:
numeric_cols=list(X_train.select_dtypes('number'))

In [111]:
# Scaling of inputs
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler().fit(X_train,Y_train)
X_train[numeric_cols]=scaler.transform(X_train[numeric_cols])
X_val[numeric_cols]=scaler.transform(X_val[numeric_cols])
X_test[numeric_cols]=scaler.transform(X_test[numeric_cols])

In [112]:
X_train.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Pregnancies,1.000000,0.123000,0.189838,-0.061880,-0.024996,0.016315,-0.071887,0.580165
Glucose,0.123000,1.000000,0.146481,0.091438,0.334260,0.284250,0.179688,0.222987
BloodPressure,0.189838,0.146481,1.000000,0.194712,0.098417,0.310252,0.031197,0.211325
SkinThickness,-0.061880,0.091438,0.194712,1.000000,0.439563,0.383551,0.218880,-0.085329
Insulin,-0.024996,0.334260,0.098417,0.439563,1.000000,0.210639,0.262018,-0.038414
BMI,0.016315,0.284250,0.310252,0.383551,0.210639,1.000000,0.153818,0.008408
DiabetesPedigreeFunction,-0.071887,0.179688,0.031197,0.218880,0.262018,0.153818,1.000000,0.033208
Age,0.580165,0.222987,0.211325,-0.085329,-0.038414,0.008408,0.033208,1.000000


Base Models

Hyperparameter tuning using Grid Search

In [113]:
cv=StratifiedKFold(n_splits=10,random_state=756,shuffle=True)
def grid_search(model,space):
  from sklearn.model_selection import GridSearchCV, StratifiedKFold
  result=GridSearchCV(model,space,scoring='accuracy',n_jobs=-1,cv=cv).fit(X_train,Y_train)
  print(f"Best Parameters: {result.best_params_}")
  print(f'Best Scores : {result.best_score_}')

Logistic Regression


In [114]:
# logistic regression
from sklearn.linear_model import LogisticRegression
logistic_regression=LogisticRegression(random_state=756,).fit(X_train,Y_train)


In [115]:
def check_score(model):
  from sklearn.metrics import accuracy_score
  print(f"Model: {model}")
  print(f"Train Score: {accuracy_score(Y_train,model.predict(X_train))} | Validation Score: {accuracy_score(Y_val,model.predict(X_val))}")


In [116]:
check_score(logistic_regression)

Model: LogisticRegression(random_state=756)
Train Score: 0.7782608695652173 | Validation Score: 0.7922077922077922


In [117]:
space={
    'tol':[1e-6,1e-5,10-4],
    'max_iter':[1000,2000,3000],
    'warm_start':[False,True],
    'solver':['lbfgs','liblinear']
}

In [118]:
grid_search(logistic_regression,space)

Best Parameters: {'max_iter': 1000, 'solver': 'liblinear', 'tol': 1e-06, 'warm_start': False}
Best Scores : 0.7695652173913043


In [119]:
logistic_regression=LogisticRegression(random_state=756,max_iter=1000,solver='liblinear',tol=1e-06).fit(X_train,Y_train)


In [120]:
check_score(logistic_regression)

Model: LogisticRegression(max_iter=1000, random_state=756, solver='liblinear',
                   tol=1e-06)
Train Score: 0.7760869565217391 | Validation Score: 0.7987012987012987


Ridge Classifier

In [121]:
from sklearn.linear_model import RidgeClassifier
ridge=RidgeClassifier(random_state=756).fit(X_train,Y_train)

In [122]:
check_score(ridge)

Model: RidgeClassifier(random_state=756)
Train Score: 0.7891304347826087 | Validation Score: 0.7857142857142857


In [123]:
space={
    'alpha':[0.1,0.2,0.5,1.0,1.5,2.0,2.5],
    'max_iter':[1000,2000,3000],
    'tol':[1e-6,1e-5,10-4],
    'solver':['svd','cholesky','lsqr','sparse_cg','sag','saga']
}
grid_search(ridge,space)

Best Parameters: {'alpha': 0.5, 'max_iter': 1000, 'solver': 'svd', 'tol': 1e-06}
Best Scores : 0.7891304347826088


In [124]:
ridge=RidgeClassifier(alpha=0.5,random_state=756,max_iter=1000,tol=1e-06,solver='svd').fit(X_train,Y_train)

In [125]:
check_score(ridge)

Model: RidgeClassifier(alpha=0.5, max_iter=1000, random_state=756, solver='svd',
                tol=1e-06)
Train Score: 0.7891304347826087 | Validation Score: 0.7792207792207793


Decision Tree

In [126]:
from sklearn.tree import DecisionTreeClassifier
decision_tree=DecisionTreeClassifier(random_state=756).fit(X_train,Y_train)

In [127]:
check_score(decision_tree)

Model: DecisionTreeClassifier(random_state=756)
Train Score: 1.0 | Validation Score: 0.6493506493506493


In [ ]:
space={
    'max_depth':[None,1,2,3,4,5,6,7,8,9,10],
    'min_samples_split':[2,3,4,5,6,7,8,9,10],
    'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10],
    'max_features':[3,4,5,6,7,8,9]
}
grid_search(decision_tree,space)

In [128]:
decision_tree=DecisionTreeClassifier(random_state=756,max_features=6,min_samples_leaf=10,min_samples_split=2).fit(X_train,Y_train)

In [129]:
check_score(decision_tree)

Model: DecisionTreeClassifier(max_features=6, min_samples_leaf=10, random_state=756)
Train Score: 0.8608695652173913 | Validation Score: 0.6883116883116883


Random Forest

In [130]:
from sklearn.ensemble import RandomForestClassifier
random_forest=RandomForestClassifier(random_state=756,n_jobs=-1).fit(X_train,Y_train)

In [131]:
check_score(random_forest)

Model: RandomForestClassifier(n_jobs=-1, random_state=756)
Train Score: 1.0 | Validation Score: 0.7077922077922078


In [35]:
space={
    'random_state':[756],
    'n_jobs':[-1],
    'max_depth':[1,2,3,4,5,6,7,8,9,10],
    'min_samples_split':[2,3,4,5,6,7,8,9,10],
    'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10],
    'max_features':[3,4,5,6,7,8,9],
    'n_estimators':[100,200,300,500],
    'max_leaf_nodes':[2,3,4,5]
}

In [ ]:
grid_search(random_forest,space)

In [132]:
random_forest=RandomForestClassifier(n_estimators=800,max_depth=7,max_features='sqrt',warm_start=True,max_samples=100,random_state=756,n_jobs=-1,min_samples_leaf=10,min_samples_split=2).fit(X_train,Y_train)

In [133]:
check_score(random_forest)

Model: RandomForestClassifier(max_depth=7, max_samples=100, min_samples_leaf=10,
                       n_estimators=800, n_jobs=-1, random_state=756,
                       warm_start=True)
Train Score: 0.8108695652173913 | Validation Score: 0.7337662337662337


SGD Classifier

In [152]:
from sklearn.linear_model import SGDClassifier
sgd_classifier=SGDClassifier(n_jobs=-1,random_state=756).fit(X_train,Y_train)

In [153]:
check_score(sgd_classifier)

Model: SGDClassifier(n_jobs=-1, random_state=756)
Train Score: 0.7847826086956522 | Validation Score: 0.7857142857142857


In [154]:
space={
    'penalty':['l2','l1','elasticnet'],
    'alpha':[0.0001,0.001,0.01,0.1,1.0],
    'max_iter':[500,1000,2000],
    'tol':[1e-6,1e-5,1e-7],
    'warm_start':[False,True],
    'learning_rate':['constant','optimal','invscaling','adaptive'],
    'average':[False,True]
}

In [155]:
grid_search(sgd_classifier,space)

Best Parameters: {'alpha': 0.001, 'average': False, 'learning_rate': 'optimal', 'max_iter': 500, 'penalty': 'l1', 'tol': 1e-06, 'warm_start': False}
Best Scores : 0.7847826086956523


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning:


16200 fits failed out of a total of 21600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
16200 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_stochastic_gradient.py", line 930, in fit
    se

In [138]:
sgd_classifier=SGDClassifier(n_jobs=-1,random_state=756,alpha=0.001,max_iter=1000,penalty='l1',tol=1e-06,warm_start=False).fit(X_train,Y_train)

In [151]:
check_score(sgd_classifier)

Model: SGDClassifier(alpha=0.001, n_jobs=-1, penalty='l1', random_state=756, tol=1e-06)
Train Score: 0.7891304347826087 | Validation Score: 0.7662337662337663


XG Boost

In [144]:
from xgboost import XGBClassifier
xgb=XGBClassifier(random_state=756,n_jobs=-1,n_estimators=100,max_depth=8,learning_rate=0.6,gamma=0.6).fit(X_train,Y_train)

In [145]:
check_score(xgb)

Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0.6, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.6, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=-1, num_parallel_tree=None, ...)
Train Score: 0.9891304347826086 | Validation Score: 0.7077922077922078


In [146]:
space={
    'n_estimators':[100,200,300,500],
    'max_depth':[1,2,3,4,5,6,7,8,9,10],
    'learning_rate':[1,0.5,0.2,0.1,0.05,0.01],
    'gamma':[0,0.2,0.4,0.6,0.7,0.9],

}

In [ ]:
grid_search(xgb,space)

Cross Validation

In [147]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=2,random_state=756)
scores=[]
for train_idx,test_idx in cv.split(X_train,Y_train):
  X_train_cv,Y_train_cv=X_train.iloc[train_idx],Y_train.iloc[train_idx]
  X_test_cv,Y_test_cv=X_train.iloc[test_idx],Y_train.iloc[test_idx]
  sgd_classifier.fit(X_train_cv,Y_train_cv)
  score = cross_val_score(sgd_classifier,X_train_cv,Y_train_cv,n_jobs=-1,cv=cv,scoring='accuracy')
  scores.append(score)
print(f"Validation score (Average): {np.mean(scores)}")


Validation score (Average): 0.7720412311265968


Not worth infact it decreases the scores

Stacking

In [168]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import StratifiedKFold
cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=756)
level0=[('random_forest', random_forest),('xgb', xgb),('ridge',ridge)]
level1=sgd_classifier
stacked=StackingClassifier(estimators=level0, final_estimator=level1, cv=cv, n_jobs=-1).fit(X_train,Y_train)

In [169]:
check_score(stacked)

Model: StackingClassifier(cv=StratifiedKFold(n_splits=10, random_state=756, shuffle=True),
                   estimators=[('random_forest',
                                RandomForestClassifier(max_depth=7,
                                                       max_samples=100,
                                                       min_samples_leaf=10,
                                                       n_estimators=800,
                                                       n_jobs=-1,
                                                       random_state=756,
                                                       warm_start=True)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=No...
                                         



---
# After Implementing multiple techniques I got the best result from StackingClassifier after tuning some hyperparameters of level0 as well as level1 models using GridSearch.
---
Model: StackingClassifier(cv=StratifiedKFold(n_splits=10, random_state=756, shuffle=True),
                   estimators=[('random_forest',
                                RandomForestClassifier(max_depth=7,
                                                       max_samples=100,
                                                       min_samples_leaf=10,
                                                       n_estimators=800,
                                                       n_jobs=-1,
                                                       random_state=756,
                                                       warm_start=True)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=No...
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=8,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=100, n_jobs=-1,
                                              num_parallel_tree=None, ...)),
                               ('ridge',
                                RidgeClassifier(alpha=0.5, max_iter=1000,
                                                random_state=756, solver='svd',
                                                tol=1e-06))],
                   final_estimator=SGDClassifier(n_jobs=-1, random_state=756),
                   n_jobs=-1)

Train Score: 0.7326086956521739

Validation Score: 0.7922077922077922




In [170]:
check_score(stacked)

Model: StackingClassifier(cv=StratifiedKFold(n_splits=10, random_state=756, shuffle=True),
                   estimators=[('random_forest',
                                RandomForestClassifier(max_depth=7,
                                                       max_samples=100,
                                                       min_samples_leaf=10,
                                                       n_estimators=800,
                                                       n_jobs=-1,
                                                       random_state=756,
                                                       warm_start=True)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=No...
                                         

In [83]:
# saving sgd_classifier model into disk

In [172]:
import joblib
joblib.dump(stacked,'Diabetes-Prediction.pkl')

['Diabetes-Prediction.pkl']